In [1]:
# General imports
import pandas as pd
import os

# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

os.chdir('../')

In [2]:
df_general = pd.read_excel('data/raw/maintenance_data_raw.xlsx')

df_general.head()

,Equipos,Horómetro,Fecha_Actual,Hora_de_Detención,Hora_de_Entrega,Tiempo_FS,Nro_Personas,Horas_Persona,Sistema,Sub_Sistemas,Tipo_de_Detención,Trabajo_Ejecutado,Nº_OT,Nº_Backlog,Status_Final,Trabajos_Pendientes,Jefe_de_Turno,Turnos
0,T_09,76888,2020-01-01,08:20:00,12:55:00,4.58,2,9.16,T_Hidraulico,Mangueras,Falla,fuga hidráulica- equipo presenta fuga en mangu...,338740,NaN,Operativo,NaN,Victor Leon,Dia
1,T_09,76888,2020-01-02,08:20:00,12:55:00,4.58,2,9.16,T_Hidraulico,Mangueras,Falla,fuga hidráulica- equipo presenta fuga en mangu...,338740,NaN,Operativo,NaN,Hernán Veliz,Dia
2,T_09,76890,2020-01-02,14:35:00,18:11:00,3.60,3,10.80,T_Electrico,Control,Falla,alarmas activas- sistema freno- se eliminan a...,0,NaN,Operativo,NaN,Hernán Veliz,Dia
3,T_09,76891,2020-01-06,20:50:00,21:05:00,0.25,3,0.75,T_Neumaticos,Inspección,Stand-By,"se realiza chequeo de neumáticos presiones, te...",0,NaN,Operativo,NaN,Cesar Nilo,Dia
4,T_09,76895,2020-01-07,09:00:00,11:25:00,2.42,0,0.00,T_Neumaticos,Posicion_3,Falla,operador informa por neumatico pos 3 en rojo. ...,0,NaN,Operativo,m/o varilla de nivel de grasa en mal estado,Patricio Cerda,Dia


In [3]:
df_general.Fecha_Actual = pd.to_datetime(df_general.Fecha_Actual, format='%d/%m/%Y %H:%M:%S')

df_general.loc[:, 'weekYear'] = df_general.Fecha_Actual.dt.strftime('%Y-%U')

df_count = df_general.groupby(['weekYear', 'Equipos'], as_index=False).size()
df_count.head()

,weekYear,Equipos,size
0,2020-00,T_09,3
1,2020-00,T_11,7
2,2020-00,T_13,2
3,2020-00,T_14,2
4,2020-00,T_17,5


In [4]:
df_toplot = df_count.groupby(['weekYear'], as_index=False)['size'].sum()

fig = go.Figure(
    data=[
        go.Bar(
            x=df_toplot['weekYear'],
            y=df_toplot['size'],
            text=df_toplot['size'],
            textposition='auto',
            marker_color='#636efa'
        )
    ]
)
fig.update_layout(
    title='Number of Maintenance Records per Week and Equipment',
    xaxis_title='Week Year',
    yaxis_title='Number of Records',
    xaxis_tickangle=-45,
    yaxis=dict(range=[0, df_toplot['size'].max()])
)
fig.show()

In [5]:
df_toplot = df_count.groupby(['Equipos'], as_index=False)['size'].sum()

fig = go.Figure(
    data=[
        go.Bar(
            x=df_toplot['Equipos'],
            y=df_toplot['size'],
            text=df_toplot['size'],
            textposition='auto',
            marker_color='#636efa'
        )
    ]
)
fig.update_layout(
    title='Number of Maintenance Records per Week and Equipo',
    xaxis_title='Equipo',
    yaxis_title='Number of Records',
    xaxis_tickangle=-45,
    yaxis=dict(range=[0, df_toplot['size'].max()])
)
fig.show()

In [6]:
df_general.loc[:, 'quarterYear'] = df_general.Fecha_Actual.dt.to_period('Q').astype(str)

df_count = df_general.groupby(['quarterYear', 'Equipos'], as_index=False).size()

df_unit = df_count.pivot(index='Equipos', columns='quarterYear', values='size').fillna(0)

fig = go.Figure(
    data=[
        go.Heatmap(
            z=df_unit.values,
            x=df_unit.columns,
            y=df_unit.index,
            colorscale='Bluered',
            colorbar=dict(title='Number of Records')
        )
    ]
)
fig.update_layout(
    title='Number of Maintenance Records per Unit and Quarter',
    xaxis_title='Quarter Year',
    yaxis_title='Unit (Equipos)',
    xaxis_tickangle=-45
)
fig.show()


In [7]:
df_general.loc[:, 'len_comment'] = df_general.Trabajo_Ejecutado.apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

fig = px.histogram(
    df_general,
    x='len_comment',
    title='Length of Comments in Maintenance Records',
    labels={'len_comment': 'Length of Comment'},
    nbins=50,
    color_discrete_sequence=['#636efa']
)

fig.show()